In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest

In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# TODO FOR NOW exclude_null_val_cols is separate from build_sql_usg_args just because it is 
# an important parameter.  However, it is simply placed in build_sql_usg_args, so it doesn't
# need to be explicitly included in the function signature.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def build_annual_kwh_sql_statement(
    cols_of_interest_agg, 
    groupby_cols, 
    agg_cols_and_types, 
    aep_derived_uoms_and_idntfrs=None, 
    build_sql_usg_args={}, 
    exclude_null_val_cols=['value'], 
    build_net_kwh_args={}, 
    include_counts_including_null=True
):
    #-------------------------
    assert(Utilities.is_object_one_of_types(build_net_kwh_args, [bool, dict]))
    #-------------------------
    if build_sql_usg_args is None:
        build_sql_usg_args = {}
    build_sql_usg_args['exclude_null_val_cols'] = exclude_null_val_cols
    #-------------------------
    # If build_net_kwh_args is False or an empty dict, the net kWh will not be calculated
    if not build_net_kwh_args:
        usg_sql_agg = AMINonVee_SQL.build_sql_usg(
            cols_of_interest=cols_of_interest_agg, 
            aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
            agg_cols_and_types=agg_cols_and_types, 
            include_counts_including_null=include_counts_including_null, 
            groupby_cols=groupby_cols, 
            **build_sql_usg_args
        )
        return usg_sql_agg.get_sql_statement()
    #-------------------------
    else:
        dflt_build_net_kwh_args = dict(
            cols_of_interest=TableInfos.AMINonVee_TI.std_columns_of_interest, 
            additional_derived_uoms=None, 
            run_careful=True, 
            value_col='value', 
            return_statement=True, 
            final_table_alias='USG', 
            insert_n_tabs_to_each_line=0, 
            prepend_with_to_stmnt=True, 
            join_mp_args=False, 
            allow_kwh_in_additional_derived_uoms=False
        )
        if isinstance(build_net_kwh_args, bool):
            build_net_kwh_args = dflt_build_net_kwh_args
        elif isinstance(build_net_kwh_args, dict):
            build_net_kwh_args = Utilities_sql.supplement_dict_with_default_values(build_net_kwh_args, dflt_build_net_kwh_args)
        else:
            assert(0)
        #-------------------------
        if aep_derived_uoms_and_idntfrs is None:
            build_net_kwh_args['additional_derived_uoms']='ALL'
        else:
            build_net_kwh_args['additional_derived_uoms']=aep_derived_uoms_and_idntfrs
        usg_sql_stmnt = AMINonVee_SQL.build_net_kwh_usage_sql_statement(
            **build_net_kwh_args, 
            **build_sql_usg_args
        )
        #-------------------------
        usg_sql_agg = AMINonVee_SQL.build_sql_usg(
            cols_of_interest=cols_of_interest_agg, 
            groupby_cols=groupby_cols, 
            agg_cols_and_types=agg_cols_and_types, 
            include_counts_including_null=include_counts_including_null, 
            schema_name=None, 
            table_name=build_net_kwh_args['final_table_alias']
        )
        usg_sql_agg_stmnt = usg_sql_agg.get_sql_statement(insert_n_tabs_to_each_line=0, include_alias=False)    
        #-------------------------
        full_sql_statement = f'{usg_sql_stmnt}\n\n{usg_sql_agg_stmnt}'
        #-------------------------
        return full_sql_statement

# ------------------------------------------------------------------------------------------
# EXAMPLES
# ------------------------------------------------------------------------------------------

In [ ]:
run_examples = False

In [ ]:
if run_examples:
    cols_of_interest_usage = [
        'serialnumber', 
        'aep_premise_nb', 
        'starttimeperiod', 
        'endtimeperiod', 
        'aep_endtime_utc', 
        'timezoneoffset', 
        'aep_derived_uom', 
        'aep_srvc_qlty_idntfr', 
        'value', 
        'aep_opco', 
        'aep_usage_dt']
    #-------------------------
    cols_of_interest_usage_agg = [
        'serialnumber', 
        'aep_premise_nb', 
        'aep_derived_uom ', 
        'aep_srvc_qlty_idntfr', 
        'value', 
        dict(field_desc='YEAR(aep_usage_dt)', alias='aep_usage_year'), 
        dict(field_desc='MONTH(aep_usage_dt)', alias='aep_usage_month')]
    groupby_cols = ['YEAR(aep_usage_dt)', 'MONTH(aep_usage_dt)', 'serialnumber', 'aep_premise_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']

    agg_cols_and_types = {'value':['sum', 'sq_sum', 'mean', 'std', 'count', 'max']}
    #aep_derived_uoms_and_idntfrs=['KWH']
    aep_derived_uoms_and_idntfrs=None
    serial_numbers = ['190700285', '190700284', '190706152', '539409516', '190043736',
                      '533116153', '532338313', '534956137', '882458469', '882458470']
    aep_premise_nbs = None
    date_range = ['2021-01-01', '2021-04-01']
    aep_opco = 'oh'

In [ ]:
if run_examples:
    sql = build_annual_kwh_sql_statement(
        cols_of_interest_usage_agg, 
        groupby_cols, 
        agg_cols_and_types, 
        aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
        build_sql_usg_args=dict(
            serial_numbers=serial_numbers, 
            date_range=date_range, 
            aep_opco=aep_opco
        ), 
        exclude_null_val_cols=None, 
        build_net_kwh_args=False, 
        include_counts_including_null=True
    )
    print(sql)

In [ ]:
if run_examples:
    sql_exclude_null = build_annual_kwh_sql_statement(
        cols_of_interest_usage_agg, 
        groupby_cols, 
        agg_cols_and_types, 
        aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
        build_sql_usg_args=dict(
            serial_numbers=serial_numbers, 
            date_range=date_range, 
            aep_opco=aep_opco
        ), 
        exclude_null_val_cols=['value'], 
        build_net_kwh_args=False, 
        include_counts_including_null=True
    )
    print(sql_exclude_null)

In [ ]:
if run_examples:
    sql_w_net_exclude_null = build_annual_kwh_sql_statement(
        cols_of_interest_usage_agg, 
        groupby_cols, 
        agg_cols_and_types, 
        aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
        build_sql_usg_args=dict(
            serial_numbers=serial_numbers, 
            date_range=date_range, 
            aep_opco=aep_opco
        ), 
        exclude_null_val_cols=['value'], 
        build_net_kwh_args=dict(
            cols_of_interest=cols_of_interest_usage, 
            allow_kwh_in_additional_derived_uoms=False
        ), 
        include_counts_including_null=True
    )
    print(sql_w_net_exclude_null)

In [ ]:
if run_examples:
    sql_w_net_exclude_null_allow_kwh = build_annual_kwh_sql_statement(
        cols_of_interest_usage_agg, 
        groupby_cols, 
        agg_cols_and_types, 
        aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
        build_sql_usg_args=dict(
            serial_numbers=serial_numbers, 
            date_range=date_range, 
            aep_opco=aep_opco
        ), 
        build_net_kwh_args=dict(
            cols_of_interest=cols_of_interest_usage, 
            allow_kwh_in_additional_derived_uoms=True
        ), 
        exclude_null_val_cols=['value'], 
        include_counts_including_null=True
    )
    print(sql_w_net_exclude_null_allow_kwh)

In [ ]:
if run_examples:
    conn_aws = Utilities.get_athena_prod_aws_connection()
    start = time.time()
    example_df = GenAn.build_df_general(
        conn_db=conn_aws, 
        build_sql_function=build_annual_kwh_sql_statement, 
        build_sql_function_kwargs=dict(
            cols_of_interest_agg = cols_of_interest_usage_agg, 
            groupby_cols = groupby_cols, 
            agg_cols_and_types = agg_cols_and_types, 
            aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
            build_sql_usg_args=dict(
                serial_numbers=serial_numbers, 
                aep_premise_nbs=aep_premise_nbs, 
                date_range=date_range, 
                aep_opco=aep_opco
            ), 
            build_net_kwh_args=False, 
            exclude_null_val_cols=None, 
            include_counts_including_null=True, 
            field_to_split='serial_numbers', 
            field_to_split_location_in_kwargs=['build_sql_usg_args', 'serial_numbers'], 
            batch_size=50, verbose=True, n_update=10
        )
    )
    total_time = time.time() - start
    print(f'total_time = {total_time}')
    example_df

# ------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

# BELOW TAKEN FROM dovs_and_summary_usage.ipynb

In [ ]:
save_dfs_to_csv = False
read_dfs_from_csv = True
save_dir_csvs = os.path.join(Utilities.get_local_data_dir(), r'dovs_and_summary_usage_data')

new_save_dir_csvs = r'W:\jesse_test\Data\dovs_and_summary_usage_data'

save_figs = False
#save_dir_base_figs = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures'
save_dir_base_figs = r'C:\Users\s346557\Documents\Presentations\OutagePredictions\MeetingWithEngineersEtc_20220127\Figures'

assert(save_dfs_to_csv+read_dfs_from_csv <=1) # Should never both read and write!

In [ ]:
fig_num = 0

In [ ]:
if not read_dfs_from_csv:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
date_0_no_outg = '2020-01-01'
date_1_no_outg = '2022-03-31'

# date_0_no_outg = '2019-01-01'
# date_1_no_outg = '2019-06-30'

In [ ]:
date_0_outg = '2020-01-01'
date_1_outg = '2022-03-31'

# date_0_outg = '2019-01-01'
# date_1_outg = '2019-06-30'

# -----------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------

# Build no outage collection and df_mp_all
where df_mp_all is a DataFrame containing all meter premise data (from OH)

In [ ]:
cols_of_interest_met_prem = [
    'mfr_devc_ser_nbr',
    'state_cd', 
    'prem_nb',
    'trsf_pole_nb',
    'annual_kwh',
    'annual_max_dmnd', 
    'mtr_stat_cd',
    'mtr_stat_cd_desc', 
    'devc_stat_cd', 
    'devc_stat_cd_desc'
]

# Find transformers which haven't experienced an outage since date_0_no_outg

### First, find all transformers which HAVE experienced an outage

In [ ]:
sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
    mjr_mnr_cause=None, 
    include_premise=False, 
    date_range=[date_0_no_outg, date_1_no_outg], 
    states='OH'
)
sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
print(sql_stmnt_outage_location_ids)

In [ ]:
if read_dfs_from_csv:
    df_outage_location_ids_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_location_ids_OG.csv'), dtype=str)
else:
    df_outage_location_ids_OG = pd.read_sql_query(sql_stmnt_outage_location_ids, conn_outages)
print(df_outage_location_ids_OG.shape)
df_outage_location_ids = df_outage_location_ids_OG.copy()

### Now, find those which HAVEN'T experienced an outage

### When df_outage_location_ids is large
i.e., when the date range used to find outages is large
<br> It is actually faster to grab ALL meters from default.meter_premise and then filter using pandas
<br> as opposed to using NOT IN and filtering with SQL

In [ ]:
sql_mp_all = MeterPremise.build_sql_meter_premise(cols_of_interest_met_prem, states=['OH']).get_sql_statement()
print(sql_mp_all)

In [ ]:
if read_dfs_from_csv:
    df_mp_all_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_mp_all_OG.csv'), dtype=str)
else:
    df_mp_all_OG = pd.read_sql(sql_mp_all, conn_aws) 
df_mp_all_OG = clm.remove_prepend_from_columns_in_df(df_mp_all_OG)
print(df_mp_all_OG.shape)
df_mp_all = df_mp_all_OG.copy()

In [ ]:
df_mp_no_outg_OG = df_mp_all[~df_mp_all['trsf_pole_nb'].isin(df_outage_location_ids['LOCATION_ID'].tolist())]
print(df_mp_no_outg_OG.shape)
df_mp_no_outg = df_mp_no_outg_OG.copy()

In [ ]:
df_mp_no_outg = Utilities_df.convert_col_types(
    df=df_mp_no_outg, 
    cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

In [ ]:
sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
    mjr_mnr_cause=None, 
    include_premise=True, 
    date_range=[date_0_outg, date_1_outg], 
    states='OH'
).get_sql_statement()
print(sql_outage_full)

In [ ]:
if read_dfs_from_csv:
    df_outage_OG = pd.read_csv(os.path.join(save_dir_csvs, 'df_outage_OG.csv'), dtype=str)
    csv_cols_and_types_to_convert_dict = {'CI_NB':np.int32, 'CMI_NB':np.float64, 'OUTG_REC_NB':[np.float64, np.int32]}
    df_outage_OG = Utilities_df.convert_col_types(df_outage_OG, csv_cols_and_types_to_convert_dict)
else:
    df_outage_OG = pd.read_sql_query(sql_outage_full, conn_outages, dtype={'CI_NB':np.int32, 
                                                                           'CMI_NB':np.float64, 
                                                                           'OUTG_REC_NB':np.int32})

# Merge df_outage with df_mp_all and build subsets (e.g., df_outage_prim, df_outage_dl_eqf, etc.) 

In [ ]:
#TODO why are prem_nb in df_mp_all all integers?  Whereas PREMISE_NB in df_outage
# are strings, and can be purely numerical, a alphanumeric, or all letters
if read_dfs_from_csv:
    df_mp_all['prem_nb'] = df_mp_all['prem_nb'].astype(str)

In [ ]:
df_outage = df_outage_OG.copy()
df_outage = df_outage.merge(df_mp_all, how='left', 
                            left_on='PREMISE_NB', right_on='prem_nb')
df_outage = Utilities_df.convert_col_types(
    df=df_outage, 
    cols_and_types_dict={'annual_kwh':float, 'annual_max_dmnd':float}, 
    to_numeric_errors='coerce', 
    inplace=True
)
# Below, 'prim' stands for primary, and means the meters are connected directly to a transformer pole causing an outage
df_outage_prim = df_outage[df_outage['LOCATION_ID']==df_outage['trsf_pole_nb']].copy()

# Below, 'prim_strict' stands for primary strict, and means the  meters are connected directly 
# to a transformer pole causing an outage, AND the equipment type causing the outage is a transformer
# (exact EQUIP_TYP_NMs given in xfmr_equip_typ_nms_of_interest)
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']
df_outage_prim_strict = df_outage[(df_outage['LOCATION_ID']==df_outage['trsf_pole_nb']) & 
                                  (df_outage['EQUIP_TYP_NM'].isin(xfmr_equip_typ_nms_of_interest))].copy()

In [ ]:
df_outage_prim_strict['mfr_devc_ser_nbr'].nunique()

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
#TODO Should I use df_outage...['prem_nb'] too?
# If so, should it be OR or AND to serial numbers?
sns_outg             = df_outage['mfr_devc_ser_nbr'].unique().tolist()
sns_outg_prim        = df_outage_prim['mfr_devc_ser_nbr'].unique().tolist()
sns_outg_prim_strict = df_outage_prim_strict['mfr_devc_ser_nbr'].unique().tolist()

In [ ]:
cols_of_interest_usage = [
    'serialnumber', 
    'aep_premise_nb', 
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset', 
    'aep_derived_uom', 
    'aep_srvc_qlty_idntfr', 
    'value', 
    'aep_opco', 
    'aep_usage_dt']
#-------------------------
cols_of_interest_usage_agg_dflt = [
    'serialnumber', 
    'aep_premise_nb', 
    'aep_derived_uom ', 
    'aep_srvc_qlty_idntfr', 
    'value', 
    dict(field_desc='YEAR(aep_usage_dt)', alias='aep_usage_year')]
#-------------------------
#aep_derived_uoms_and_idntfrs=['KWH']
aep_derived_uoms_and_idntfrs=None
#-------------------------
groupby_cols_dflt = ['YEAR(aep_usage_dt)', 'serialnumber', 'aep_premise_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']
agg_cols_and_types = {'value':['sum', 'sq_sum', 'mean', 'std', 'count', 'max']}
sort_by_dflt = ['aep_usage_year', 'serialnumber', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']

In [ ]:
#date_range = ['2021-01-01', '2021-04-01']
date_range = [date_0_outg, date_1_outg]

include_groupby_month = True
include_groupby_day = False
serial_numbers=['883464195', '883463751', '883464198', '883463775', '883463138', '883463776']
aep_premise_nbs = None
aep_opco='oh'


# include_groupby_month = True
# include_groupby_day = False
# serial_numbers=None
# aep_premise_nbs=['070508983','071048983','071108983','072018983','073608983','074148983','074208983',
#                  '075738983','075897983','076708983','077308983','078838983','079808983']
# aep_opco='oh'


include_groupby_month = True
include_groupby_day = False
serial_numbers = sns_outg_prim_strict
aep_premise_nbs = None
aep_opco='oh'

In [ ]:
#batch_size = 50
#batch_size = 1000
batch_size = 10
verbose = True
#n_update = 10
n_update = 1

save_args = dict(save_to_file=True, 
                 save_dir = os.path.join(Utilities_config.get_local_data_dir(), r'build_annual_kwh'), 
                 save_name=r'annual_kwh_prim_strict.csv')

In [ ]:
cols_of_interest_usage_agg = copy.deepcopy(cols_of_interest_usage_agg_dflt)
groupby_cols               = copy.deepcopy(groupby_cols_dflt)
sort_by                    = copy.deepcopy(sort_by_dflt)

if include_groupby_month:
    sort_by.insert(1, 'aep_usage_month')
    cols_of_interest_usage_agg.append(dict(field_desc='MONTH(aep_usage_dt)', alias='aep_usage_month'))
    groupby_cols.insert(1, 'MONTH(aep_usage_dt)')
    
if include_groupby_day:
    assert(include_groupby_month)
    sort_by.insert(2, 'aep_usage_day')
    cols_of_interest_usage_agg.append(dict(field_desc='DAY(aep_usage_dt)', alias='aep_usage_day'))
    groupby_cols.insert(2, 'DAY(aep_usage_dt)')

In [ ]:
start = time.time()
test_df = GenAn.build_df_general(
    conn_db=conn_aws, 
    build_sql_function=build_annual_kwh_sql_statement, 
    build_sql_function_kwargs=dict(
        cols_of_interest_agg = cols_of_interest_usage_agg, 
        groupby_cols = groupby_cols, 
        agg_cols_and_types = agg_cols_and_types, 
        aep_derived_uoms_and_idntfrs=aep_derived_uoms_and_idntfrs, 
        build_sql_usg_args=dict(
            serial_numbers=serial_numbers, 
            aep_premise_nbs=aep_premise_nbs, 
            date_range=date_range, 
            aep_opco=aep_opco
        ), 
        build_net_kwh_args=dict(
            cols_of_interest=cols_of_interest_usage, 
            allow_kwh_in_additional_derived_uoms=True
        ), 
        exclude_null_val_cols=None, 
        include_counts_including_null=True, 
        field_to_split='serial_numbers', 
        field_to_split_location_in_kwargs=['build_sql_usg_args', 'serial_numbers'], 
        batch_size=batch_size, verbose=verbose, n_update=n_update
    ), 
    save_args=save_args
)
total_time = time.time() - start
print(f'total_time = {total_time}')